In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import netCDF4
from netCDF4 import Dataset
import math
import os
import datetime as dt

In [ ]:
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)

## Meteorological Data

In [ ]:
df = pd.read_csv('HistoricalData/HistMetData_46035.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.rename(columns={'Unnamed: 0':'DateTime'},inplace=True)
df.head()

In [ ]:
df.replace([99.0,999.0, 9999.0],np.NAN,inplace=True)

In [ ]:
df.describe()

In [ ]:
df.drop(columns=['VIS','TIDE'],axis=1,inplace=True)

In [ ]:
df.describe()

In [ ]:
df.isna().any()

#### All the columns have missing data

In [ ]:
msno.bar(df)

In [ ]:
df.drop(columns=['MWD','DEWP'],axis=1,inplace=True)

In [ ]:
msno.bar(df)

In [ ]:
df_dropped = df.dropna()
df_dropped.describe()

In [ ]:
df_dropped.shape

## Ocean Currents Data

In [7]:
#Reading list of stations with available currents data
stations = pd.read_csv('19_stations.csv',skipinitialspace=True)

#Converting Longitude west to east, to match with currents data notation
stations['lon'] = 360-stations['lon']

# Creating a new dataset to store all currents data
test = stations.copy()

#Pivoting the 'Station' column to make each station a column name
test = pd.pivot_table(test, columns=['Station'])

#dropping longitude & latitude values. Instead, we'll use original 'Stations' table for the longitude and latitudes
test.drop(test.index, inplace=True)

#Adding 19 new columns to make 38 multi-index columns (2 for each of the 19 stations)
test = pd.concat([test,test.reindex(columns=list(range(19)))])
columns=stations['Station'].values
velocities = ['u','v']

# Creating multi-index columns, u, v for each of the 19 stations
test.columns=pd.MultiIndex.from_product([columns,velocities])

test

Empty DataFrame
Columns: [(46015, u), (46015, v), (46027, u), (46027, v), (46022, u), (46022, v), (46014, u), (46014, v), (PTAC1, u), (PTAC1, v), (46002, u), (46002, v), (46013, u), (46013, v), (46026, u), (46026, v), (46012, u), (46012, v), (46042, u), (46042, v), (46028, u), (46028, v), (46011, u), (46011, v), (PTGC1, u), (PTGC1, v), (46054, u), (46054, v), (46053, u), (46053, v), (46069, u), (46069, v), (46025, u), (46025, v), (46047, u), (46047, v), (46086, u), (46086, v)]
Index: []

[0 rows x 38 columns]

In [8]:
for fileName in os.listdir('CurrentsData/'):
    fileDate = fileName[-13:-3]
    try:
        dataset = Dataset('CurrentsData/'+fileName)
        u = dataset.variables['u']
        v = dataset.variables['v']
        lon_array = dataset.variables['lon'][:]
        lat_array = dataset.variables['lat'][:]
        time_array = dataset.variables['time'][:]
        depth_array = dataset.variables['depth'][:]

        test = test.append(pd.Series(name=fileDate))
        for i, row in stations.iterrows():

            lat_index = np.abs(lat_array - row['lat']).argmin()
            lon_index = np.abs(lon_array - row['lon']).argmin()
            dep_index = np.abs(depth_array - 0).argmin()
            time_index = np.abs(time_array - 1.0).argmin()

            if(lat_array[lat_index] - row['lat'] <= 0.1):
                u_value = u[:,dep_index,lat_index,lon_index]
            else:
                u_value = np.NAN

            if(lon_array[lon_index] - row['lon'] <= 0.1):
                v_value = v[:,dep_index,lat_index,lon_index]
            else:
                v_value = np.NAN

            test.at[fileDate, (row['Station'],'u')] = u_value
            test.at[fileDate, (row['Station'],'v')] = v_value
    except:
        print("Unable to process the file",fileName)
    finally:
        dataset.close()
    

In [9]:
test.shape

(1041, 38)

In [11]:
test.head()

46015               46027               46022            \
                   u         v         u         v         u         v   
2013010103  0.022879 -0.176103 -0.042675 -0.117706 -0.159065 -0.225282   
2013010109 -0.132579 -0.133023 -0.061303  0.168611 -0.339585 -0.352252   
2013010115  0.000021  0.156359  0.033921 -0.011597 -0.288465 -0.099120   
2013010121 -0.024741 -0.024946  0.017932 -0.037752 -0.269262 -0.288108   
2013010203 -0.026595 -0.113011  0.015125 -0.198444 -0.281528 -0.245974   

               46014               PTAC1              ...        46053  \
                   u         v         u         v    ...            u   
2013010103  0.012454 -0.596346 -0.098758  0.189663    ...     0.127786   
2013010109 -0.478630 -0.030841  0.022117  0.411831    ...    -0.080875   
2013010115  0.053899 -0.074639 -0.093680  0.019036    ...    -0.025048   
2013010121 -0.024331  0.064643 -0.048259  0.382174    ...    -0.132510   
2013010203 -0.113954  0.107819  0.028518  0.155182    ...     0.138505   

                         46069               46025               46047  \
                   v         u         v         u         v         u   
2013010103 -0.052983  0.145036 -0.018463  0.112119 -0.080323  0.257006   
2013010109 -0.017088  0.075092 -0.045910 -0.109762 -0.059036 -0.059332   
2013010115  0.029185  0.153242 -0.006796  0.018986  0.164402  0.170121   
2013010121  0.055814 -0.070026 -0.067319 -0.120629 -0.004231 -0.116132   
2013010203 -0.044682  0.162526 -0.072071  0.131868 -0.019968  0.154311   

                         46086            
                   v         u         v  
2013010103 -0.198736  0.014944 -0.174077  
2013010109 -0.177118 -0.151422 -0.059866  
2013010115 -0.068729 -0.050464  0.098153  
2013010121 -0.275085 -0.001019 -0.118416  
2013010203 -0.064177 -0.053546 -0.084577  

[5 rows x 38 columns]

In [14]:
test.isna().sum().any()

False

In [17]:
test.reset_index(col_fill='DateTime',inplace=True)
test.head()

index     46015               46027               46022            \
     DateTime         u         v         u         v         u         v   
0  2013010103  0.022879 -0.176103 -0.042675 -0.117706 -0.159065 -0.225282   
1  2013010109 -0.132579 -0.133023 -0.061303  0.168611 -0.339585 -0.352252   
2  2013010115  0.000021  0.156359  0.033921 -0.011597 -0.288465 -0.099120   
3  2013010121 -0.024741 -0.024946  0.017932 -0.037752 -0.269262 -0.288108   
4  2013010203 -0.026595 -0.113011  0.015125 -0.198444 -0.281528 -0.245974   

      46014               PTAC1    ...        46053               46069  \
          u         v         u    ...            u         v         u   
0  0.012454 -0.596346 -0.098758    ...     0.127786 -0.052983  0.145036   
1 -0.478630 -0.030841  0.022117    ...    -0.080875 -0.017088  0.075092   
2  0.053899 -0.074639 -0.093680    ...    -0.025048  0.029185  0.153242   
3 -0.024331  0.064643 -0.048259    ...    -0.132510  0.055814 -0.070026   
4 -0.113954  0.107819  0.028518    ...     0.138505 -0.044682  0.162526   

                46025               46047               46086            
          v         u         v         u         v         u         v  
0 -0.018463  0.112119 -0.080323  0.257006 -0.198736  0.014944 -0.174077  
1 -0.045910 -0.109762 -0.059036 -0.059332 -0.177118 -0.151422 -0.059866  
2 -0.006796  0.018986  0.164402  0.170121 -0.068729 -0.050464  0.098153  
3 -0.067319 -0.120629 -0.004231 -0.116132 -0.275085 -0.001019 -0.118416  
4 -0.072071  0.131868 -0.019968  0.154311 -0.064177 -0.053546 -0.084577  

[5 rows x 39 columns]

In [28]:
test['DateObserved'] = pd.to_datetime(test[('index','DateTime')].str[:4] + '/' + test[('index','DateTime')].str[4:6] + '/' + test[('index','DateTime')].str[6:8] + ' ' + test[('index','DateTime')].str[-2:]+':00:00')
test.head()

index     46015               46027               46022            \
     DateTime         u         v         u         v         u         v   
0  2013010103  0.022879 -0.176103 -0.042675 -0.117706 -0.159065 -0.225282   
1  2013010109 -0.132579 -0.133023 -0.061303  0.168611 -0.339585 -0.352252   
2  2013010115  0.000021  0.156359  0.033921 -0.011597 -0.288465 -0.099120   
3  2013010121 -0.024741 -0.024946  0.017932 -0.037752 -0.269262 -0.288108   
4  2013010203 -0.026595 -0.113011  0.015125 -0.198444 -0.281528 -0.245974   

      46014               PTAC1         ...             46053     46069  \
          u         v         u         ...                 v         u   
0  0.012454 -0.596346 -0.098758         ...         -0.052983  0.145036   
1 -0.478630 -0.030841  0.022117         ...         -0.017088  0.075092   
2  0.053899 -0.074639 -0.093680         ...          0.029185  0.153242   
3 -0.024331  0.064643 -0.048259         ...          0.055814 -0.070026   
4 -0.113954  0.107819  0.028518         ...         -0.044682  0.162526   

                46025               46047               46086            \
          v         u         v         u         v         u         v   
0 -0.018463  0.112119 -0.080323  0.257006 -0.198736  0.014944 -0.174077   
1 -0.045910 -0.109762 -0.059036 -0.059332 -0.177118 -0.151422 -0.059866   
2 -0.006796  0.018986  0.164402  0.170121 -0.068729 -0.050464  0.098153   
3 -0.067319 -0.120629 -0.004231 -0.116132 -0.275085 -0.001019 -0.118416   
4 -0.072071  0.131868 -0.019968  0.154311 -0.064177 -0.053546 -0.084577   

         DateObserved  
                       
0 2013-01-01 03:00:00  
1 2013-01-01 09:00:00  
2 2013-01-01 15:00:00  
3 2013-01-01 21:00:00  
4 2013-01-02 03:00:00  

[5 rows x 40 columns]

In [30]:
test.to_excel('Currents_Data.xlsx')

# Merging Meterological & Currents Data

### Function to read & clean Historical Meterological Data files:

In [26]:
def read_meto_file(filepath):
    try:
        #Loading the CSV file
        df_temp = pd.read_csv(filepath,skipinitialspace=True)
        #Renaming the anonymus column
        df_temp.rename(columns={'Unnamed: 0':'Date_Hist'},inplace=True)
        #Dropping unnecessary columns
        df_temp.drop(columns=['VIS','TIDE','MWD','DEWP'],axis=1,inplace=True)
        #Replacing missing values with np.Nan notation
        df_temp.replace([99.0,999.0, 9999.0],np.NAN,inplace=True)
        #Dropping missing values
        df_temp = df_temp.dropna()
        # Rounding up 'Date_Hist' to nearest hour to match it with "currents data"
        df_temp['Date_Hist'] = pd.to_datetime(df_temp['Date_Hist']).dt.round('H')
        return df_temp
    except Exception as e:
        print("Exception occurred", e.message)

In [27]:
df_46042 = read_meto_file('HistoricalData/HistMetData_46042.csv')
df_46042.head()

,Date_Hist,WDIR,WSPD,GST,WVHT,DPD,APD,PRES,ATMP,WTMP
0,2013-01-01 00:00:00,317.0,4.4,5.4,2.28,19.05,10.49,1023.7,11.2,13.0
1,2013-01-01 01:00:00,316.0,4.9,5.9,2.23,19.05,10.03,1023.9,11.0,12.9
2,2013-01-01 02:00:00,331.0,4.6,5.8,2.34,19.05,9.86,1024.0,11.0,12.8
3,2013-01-01 03:00:00,342.0,5.2,6.6,2.26,19.05,9.93,1024.2,11.1,12.8
4,2013-01-01 04:00:00,346.0,4.4,5.6,1.92,17.39,9.03,1024.4,11.1,12.7


### Function to merge Meteorological & Currents data

In [73]:
def merge_currents(df_temp, station):
    try:
        currents = pd.read_excel('Currents_Data.xlsx',header=[0,1])
        currents.reset_index(drop=True,inplace=True)
        rename_col = dict(zip(currents.columns.levels[1],['DateHour', 'u', 'v']))
        currents.rename(columns=rename_col,level=1, inplace=True)

        df_left = df_temp.set_index('Date_Hist')
        df_right = currents[[(station,'u'),(station,'v'),('DateObserved','DateHour')]].set_index(('DateObserved','DateHour'))
        df_temp_merged = df_left.join(df_right,how='left')
        df_temp_merged.rename(columns={df_temp_merged.columns[-2]:'u',df_temp_merged.columns[-1]:'v'},inplace=True)
        df_temp_merged.reset_index(inplace=True)
        return df_temp_merged
    except Exception as e:
        print("Exception occurred", e.message)
    

In [74]:
df_46042_merged = merge_currents(df_46042, '46042')

C:\Users\HPHP\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:543: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [76]:
df_46042_merged.shape
df_46042_merged.head()

(38712, 12)

,Date_Hist,WDIR,WSPD,GST,WVHT,DPD,APD,PRES,ATMP,WTMP,u,v
0,2013-01-01 00:00:00,317.0,4.4,5.4,2.28,19.05,10.49,1023.7,11.2,13.0,NaN,NaN
1,2013-01-01 01:00:00,316.0,4.9,5.9,2.23,19.05,10.03,1023.9,11.0,12.9,NaN,NaN
2,2013-01-01 02:00:00,331.0,4.6,5.8,2.34,19.05,9.86,1024.0,11.0,12.8,NaN,NaN
3,2013-01-01 03:00:00,342.0,5.2,6.6,2.26,19.05,9.93,1024.2,11.1,12.8,0.12697,-0.092093
4,2013-01-01 04:00:00,346.0,4.4,5.6,1.92,17.39,9.03,1024.4,11.1,12.7,NaN,NaN


In [77]:
df_46042_merged.to_csv('df_46042_merged.csv',index=False)

In [79]:
df_46042_clean = df_46042_merged.dropna()
df_46042_clean.shape

(1041, 12)

In [80]:
df_46042_clean

,Date_Hist,WDIR,WSPD,GST,WVHT,DPD,APD,PRES,ATMP,WTMP,u,v
3,2013-01-01 03:00:00,342.0,5.2,6.6,2.26,19.05,9.93,1024.2,11.1,12.8,0.126970,-0.092093
9,2013-01-01 09:00:00,5.0,1.4,2.2,2.50,17.39,11.68,1023.5,11.0,12.4,-0.038502,0.027108
15,2013-01-01 15:00:00,83.0,5.0,6.6,2.98,16.00,12.80,1023.0,9.7,12.4,0.155206,-0.050650
21,2013-01-01 21:00:00,81.0,5.2,6.9,2.48,16.00,10.99,1022.1,10.7,12.8,-0.075122,-0.094673
27,2013-01-02 03:00:00,25.0,5.2,6.1,2.74,16.00,10.69,1022.3,11.8,12.7,0.131738,-0.065950
33,2013-01-02 09:00:00,77.0,8.9,10.3,2.43,14.81,8.55,1021.8,10.3,12.5,0.021014,-0.073061
39,2013-01-02 15:00:00,76.0,7.3,8.7,2.13,13.79,8.65,1021.1,10.2,12.4,0.032055,0.033441
45,2013-01-02 21:00:00,101.0,5.6,6.7,1.91,16.00,8.00,1020.7,12.0,12.6,0.080874,-0.010986
51,2013-01-03 03:00:00,13.0,2.6,3.0,2.03,16.00,12.84,1020.0,12.3,12.7,0.066022,-0.111679
57,2013-01-03 09:00:00,339.0,2.5,3.1,1.73,14.81,12.26,1020.7,11.9,12.5,-0.017844,0.007497


## Archive

In [ ]:
dataset.dimensions

In [ ]:
lon = dataset.variables['lon']
lat = dataset.variables['lat']
time = dataset.variables['time']
depth = dataset.variables['depth']

for d in dataset.dimensions.items():
    print(d)

lon_array = lon[:]
lat_array = lat[:]
time_array = time[:]
depth_array = depth[:]

lon_array
lat_array
time_array
depth_array

In [ ]:
i = np.abs(lon_array - 10).argmin()
j = np.abs(lat_array - 30).argmin()
u_time = u[:,:,j,i]
u_time

In [ ]:
u[:,:,:,:]